# PySpark

Hadoop과 Spark의 차이
- Hadoop: 데이터를 분산시켜 저장하는 것이 주된 목적이다.
- Spark: 분산된 데이터를 메모리에 올려 분석, 처리하는 것이 주된 목적이다.

Spark의 특징
- 디스크에 저장된 데이터를 메모리에 올려 작업하므로 처리 속도가 빠르다.
- 사용하기 쉽다(Hadoop은 Java만 지원, Spark는 4개 언어 지원)
- Hadoop과 달리 여러 add-on 없이 Spark만으로 작업이 가능하다.
- 다양한 file system을 사용 가능하다.

Spark 구성 요소
- Driver: master node
- Worker: slave nodes

In [1]:
from pyspark import SparkContext

In [4]:
sc = SparkContext(master='local[2]', appName='Spark Test')
sc

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /Users/keith_lee/anaconda3/lib/python3.6/site-packages/IPython/utils/py3compat.py:188 

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

이미 sc라는 SparkContext가 만들어져 있기에 에러.

In [5]:
from pyspark import SparkConf

In [6]:
conf = SparkConf()
conf.setMaster('local[3]').setAppName('Conf')

In [7]:
sc.stop()

SparkContext 사용 종료.

In [8]:
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[3] appName=Conf>

설정한 conf를 사용하여 SparkContext 생성.

Spark = RDD

RDD의 특성
- Immutable.
- Lazy Execution.
- Action: 조각난 파일을 RDD로 묶어 Worker에게 지시한 후 받아오는 과정(Reduce가 대표적인 Action이다.)

In [10]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:184

RDD 객체를 생성하였다.

In [11]:
rdd.collect()

[1, 2, 3, 4, 5]

collect(): RDD의 모든 객체를 가져와 내용 출력.(RDD action의 일종)

In [12]:
rdd = sc.parallelize([1, 2, 3, 4, 5], 3)

rdd.glom().collect()

[[1], [2, 3], [4, 5]]

- parallelize: 입력된 데이터에 대해 지정한 갯수만큼 partition을 나눈다.
- glom: partition이 어떻게 나뉘어져 있는지에 대한 정보를 가져온다.(RDD transform의 일종)

In [13]:
text = sc.parallelize('~/sample/test-text.txt')

text.glom().collect()

[['~', '/', 's', 'a', 'm', 'p', 'l'],
 ['e', '/', 't', 'e', 's', 't', '-'],
 ['t', 'e', 'x', 't', '.', 't', 'x', 't']]

In [14]:
text.collect()

['~',
 '/',
 's',
 'a',
 'm',
 'p',
 'l',
 'e',
 '/',
 't',
 'e',
 's',
 't',
 '-',
 't',
 'e',
 'x',
 't',
 '.',
 't',
 'x',
 't']

str을 철자 단위로 쪼개 출력한다.

In [17]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x:x%2==0).collect()

[2, 4]

lambda expression을 사용하여 데이터를 가져온다.

In [16]:
rdd = sc.parallelize(['b', 'a', 'c'], 3)
rddMap = rdd.map(lambda x:(x, 1))

sorted(rddMap.glom().collect())

[[('a', 1)], [('b', 1)], [('c', 1)]]

In [19]:
rdd = sc.parallelize([2, 3, 4])
rdd1 = rdd.map(lambda x:range(1, x))

rdd1.glom().collect()

[[range(1, 2)], [range(1, 3)], [range(1, 4)]]

In [20]:
rdd.flatMap(lambda x:range(1, x)).collect()

[1, 1, 2, 1, 2, 3]

lambda expression을 사용하여 데이터를 mapping한다.

In [21]:
rdd = sc.parallelize(['Roses are red', 'Violets are blue'])
rdd.map(lambda x:x.split()).collect()

[['Roses', 'are', 'red'], ['Violets', 'are', 'blue']]

In [22]:
rdd.flatMap(lambda x:x.split()).collect()

['Roses', 'are', 'red', 'Violets', 'are', 'blue']

lambda expression을 사용하여 mapping한 것을 다시 풀어서 가져올 수 있다.(평면화시켜 일렬로 만드는 방법)

In [23]:
sc.parallelize([1, 1, 2, 3]).distinct().collect()

[3, 1, 2]

In [24]:
rdd.flatMap(lambda x:x.split()).distinct().collect()

['are', 'Roses', 'Violets', 'blue', 'red']

dictinct(): 중복 제거 - 데이터 정제에 유용함.

In [25]:
rdd = sc.parallelize(range(100), 4)
subset = rdd.sample(False, 0.1, 12)

subset.collect()

[8, 10, 15, 16, 19, 26, 32, 62, 63, 65, 72, 98]

In [26]:
subset.glom().collect()

[[8, 10, 15, 16, 19], [26, 32], [62, 63, 65, 72], [98]]

데이터셋에서 지정된 갯수만큼 랜덤하게 가져온다.

In [27]:
subset.count()

12

In [28]:
len(subset.collect())

12

- subset.count(): Spark 내부에서 실행하므로 시간이 덜 걸린다.
- len(subset.collect()): collect를 사용하여 Spark의 연산 결과를 list로 받고 파이썬 내부에서 len 함수를 실행하므로 시간이 더 걸린다.

In [29]:
x = sc.parallelize([('a', 1), ('b', 4)])
y = sc.parallelize([('a', 2)])

In [30]:
sorted(x.leftOuterJoin(y).collect())

[('a', (1, 2)), ('b', (4, None))]

In [31]:
sorted(y.leftOuterJoin(x).collect())

[('a', (2, 1))]

2개의 RDD에 대해 left outer join 실행.

In [32]:
sorted(x.rightOuterJoin(y).collect())

[('a', (1, 2))]

left outer join과 right outer join의 결과가 다르다.

In [33]:
sorted(x.join(y).collect())

[('a', (1, 2))]

2개의 RDD에 대해 right outer join을 실행한 결과와 동일.

In [34]:
rdd = sc.parallelize(range(100), 4)
subset = rdd.sample(False, 0.1, 12)

subset.count()

12

In [35]:
subset.collect()

[8, 10, 15, 16, 19, 26, 32, 62, 63, 65, 72, 98]

subset은 RDD이기 때문에 collect()를 써줘야 출력된다.

In [36]:
rdd.takeSample(False, 10, 123)

[26, 14, 70, 99, 60, 78, 80, 13, 33, 43]

takeSample은 action이기 때문에 따로 collect() 안써줘도 출력된다.

In [38]:
rdd = sc.parallelize([1, 2, 3, 4, 5], 2)
rdd.reduce(lambda x, y:x+y)

15

In [39]:
rdd.glom().collect()

[[1, 2], [3, 4, 5]]

Partition별로 우선 연산 실행: 1+2, 3+4를 계산한 다음 (3+4)+5를 계산한 후 (1+2)+((3+4)+5)를 계산한다.

In [40]:
rdd = sc.parallelize([1, 2, .5, .1, 5, .2], 2)
rdd.reduce(lambda x, y: x/y)
rdd.glom().collect()

[[1, 2, 0.5], [0.1, 5, 0.2]]

In [41]:
rdd = sc.parallelize([1, 2, .5, .1, 5, .2], 3)
rdd.reduce(lambda x, y: x/y)
rdd.glom().collect()

[[1, 2], [0.5, 0.1], [5, 0.2]]

Partition을 나누는 방법이 달라지면 계산 결과도 바뀐다. => repartition의 필요성(나눗셈, 뺄셈시 연산 순서 유의할것)

In [42]:
rdd = sc.parallelize([('a', 1), ('b', 2), ('a', 3)])

In [43]:
from operator import add
sorted(rdd.reduceByKey(add).collect())

[('a', 4), ('b', 2)]

key값이 같으면 value끼리 더한다.

In [44]:
rdd = sc.parallelize([('a', 1), ('b', 2), ('a', 3)], 2)

In [46]:
rdd.saveAsTextFile('abcd')

In [47]:
rdd.glom().collect()

[[('a', 1)], [('b', 2), ('a', 3)]]

Partition마다 파일이 만들어지고 Partition된 내용이 들어간다.

('a', 1) 따로, ('b', 2), ('a', 3) 따로 들어간다.

In [49]:
fromText = sc.textFile('abcd')
fromText.collect()

["('b', 2)", "('a', 3)", "('a', 1)"]

저장된 경로를 적어주면 알아서 분산 저장된 데이터를 가져온다.

In [50]:
fromText.glom().collect()

[["('b', 2)", "('a', 3)"], [], ["('a', 1)"]]

저장시 적용된 partiton과 저장된 파일을 불러오며 적용되는 partition은 다르다.

In [51]:
fromText = fromText.repartition(2)
fromText.glom().collect()

[["('b', 2)", "('a', 3)", "('a', 1)"], []]

파일을 불러와서 다시 partition 적용.

In [52]:
data = sc.parallelize(
    [('a', 22), {'b': 23}, ['c', 4]]
)

obj = data.collect()

In [53]:
type(obj)

list

In [54]:
obj[1]['b']

23

In [55]:
obj[2][1]

4

파이썬과 py4j를 오가며 작업. RDD는 jvm에서 메모리를 할당하고, 별도로 가져온 obj만 파이썬에서 메모리를 할당한다.

___

## DataFrame
- 구조적 데이터 처리를 가능하게 함.
- 실제 내부 구조는 RDD이나 RDD를 한번 더 감싸 처리하므로 속도가 빠르다.
- 특징: Immutable(읽기 전용), Lazy Execution(Action 안써주면 실행 안함)
- 장점
    * Catalyst Optimizer: 트리 구조로 RDD 표현. 논리적 plan을 최적화하여 가장 효과적인 물리적 plan 결정.
    * Tungsten: 메모리를 직접 관리 - 효율적.

In [56]:
from pyspark.sql import SparkSession

In [60]:
spark = SparkSession.builder.master('local[2]').appName('Session').getOrCreate()
spark

In [59]:
sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[3] appName=Conf>

spark라는 이름으로 session을 만들었으나 sc, spark의 driver는 똑같다. => sc가 stop되는 순간 spark도 못쓴다.

In [61]:
sc.stop()